In [ ]:
max_length = 256  # sms(가사) 최대 길이

# 1. 데이터 불러오기

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('sms.tsv', sep = '\t', )
# df.columns = ['label', 'sms']
print(df.columns)
print(df.shape)

Index(['label', 'sms'], dtype='object')
(5574, 2)


In [ ]:
df.head()

,label,sms
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,spam,"%^^×？×^×&#****,>,;//×&>>*(*^%=÷#~^&,****)"
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# 클래스 파악
classes = sorted(set(df['label']))
class_to_idx = {}

for i, c in enumerate(classes):  # 모든 클래스에 대해
    class_to_idx.update({c: i})

nclass = len(classes)

print("# of classes: %d" %nclass)
print(classes)
print(class_to_idx)

# of classes: 2
['ham', 'spam']
{'ham': 0, 'spam': 1}


# 2. 새로운 DataFrame
## 1) 'label, sms'만 남기기
## 2) 최대 텍스트 길이 만큼 자르기 #pandas.Series.str.slice
### .'성별, 가사'만 남기려면?

In [ ]:
new_df = pd.DataFrame({'label':df['label'],
                       'sms':df['sms'].str.slice(  # 최대 가사 텍스트 만큼 자르기
                          start = 0, stop = max_length)
                       })

# 3) 중복 제거

In [ ]:
len(new_df)

5574

In [ ]:
new_df = pd.DataFrame(new_df.drop_duplicates())

In [ ]:
len(new_df)

5171

# 4) 셔플

In [ ]:
df_shuffled = new_df.sample(frac = 1).reset_index(drop = True)
df_shuffled.head()

,label,sms
0,ham,is your hamster dead? Hey so tmr i meet you at...
1,ham,Hope you are not scared!
2,ham,In da car park
3,ham,Ok. But i finish at 6.
4,ham,R u still working now?


# 5) train, test 나누기

In [ ]:
# train: test = 9:1
# train: test = 540: 60 -> train:valid:test = 432:108:60
train_ratio = 0.9

# train dataset
s, e = 0, int(df_shuffled.shape[0] * train_ratio)  # # of rows
df_train = pd.DataFrame({'label':df_shuffled['label'][s:e],
                         'sms':df_shuffled['sms'][s:e]})
print("index for train: %d~%d" %(s, e))

# test dataset
s, e = e, e + int(df_shuffled.shape[0] * (1.0 - train_ratio))
print("index for test: %d~%d" %(s, e))
df_test = pd.DataFrame({'label':df_shuffled['label'][s:e],
                        'sms':df_shuffled['sms'][s:e]})

index for train: 0~4653
index for test: 4653~5170


In [ ]:
# column 수 확인
print(df_train.shape)
print(df_test.shape)

(4653, 2)
(517, 2)


# 4) 저장

In [ ]:
# new_df.columns : ['성별', '가사']
df_train.to_csv('./sms.maxlen.uniq.shuf.train.tsv',
                header = False, index = False, sep = '\t')

df_test.to_csv('./sms.maxlen.uniq.shuf.test.tsv',
               header = False, index = False, sep = '\t')

# 데이터 로더

In [14]:
import torch

print(torch.__version__)

1.9.0+cu102


In [16]:
!pip install torchtext==0.4.0

     |████████████████████████████████| 61kB 7.4MB/s 
  Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [17]:
import torchtext
import numpy as np

# 3. 데이터 로드함수

In [ ]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset = mnist_train,  # mnist_train 를 트레인 시키자.
                                           batch_size = batch_size,
                                           shuffle = True,
                                           drop_last = True)  # batch_size 만큼 나눌 때 나머지는 버려라.

test_loader = torch.utils.data.DataLoader(dataset = mnist_test,
                                          batch_size = batch_size,
                                          shuffle = False,  # test 에서는 shuffle 하지 않음.
                                          drop_last = True)

# 데이터 로드함수 이해하기

In [ ]:
n = 3  # 샘플로 그려볼 데이터 개수
for i, [imgs, labels] in enumerate(test_loader):  # batch_size 만큼
    if i > 5:
        break

    print("[%d]" %i)
    print("한 번에 로드되는 데이터 크기:", len(imgs))

    # 그리기
    for j in range(n):
        img = imgs[j].numpy()  # image 타입을 numpy 로 변환 (1, 28, 28)
        img = img.reshape((img.shape[1], img.shape[2]))  # (1, 28, 28) -> (28, 28)
        #print img.shape

        plt.subplot(1, n, j + 1)  # (1, 3) 형태 플랏의 j 번째 자리에 그리겠다.
        plt.imshow(img, cmap = 'gray')
        plt.title("label: %d" %labels[j])
    plt.show()

In [ ]:
from data_loader import DataLoader